In [1]:
from datetime import datetime
import requests
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, Range1d, Title, SaveTool
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

In [2]:
def get_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        quote_ts = datetime.now()
        return response.json(), quote_ts
    else:
        print(f"Fehler beim Laden der Daten: {response.status_code}")
        return None, None

In [3]:
data, quote_ts = get_data("https://privacytests.org/index.json")

In [4]:
def get_update_date():
    try:
        website = requests.get("https://privacytests.org/").text
        update_date = website.split(">Updated ")[1].split("</div>")[0]
        date = datetime.strptime(update_date, "%Y-%m-%d")
        return date.strftime("%d.%m.%Y")
    except Exception:
        return "n.d."

In [5]:
update_date = get_update_date()

In [6]:
def get_browser_tests(data):
    browser_tests = dict()
    for browser in data["all_tests"]:
        browser_name = browser["browser"]
        test_sections = browser["testResults"]
        section = dict()
        for section_name, tests in test_sections.items():
            test_items = dict()
            for test_name, test in tests.items():
                try:
                    unsupported = test["unsupported"]
                except KeyError:
                    unsupported = None
                    passed = test["passed"]
                if unsupported:
                    passed = False
                else:
                    try:
                        passed = test["passed"]
                    except KeyError:
                        passed = test["testFailed"]
                test_items[test_name] = passed
            section[section_name] = test_items
        browser_tests[browser_name] = section
    return browser_tests

In [7]:
browser_tests = get_browser_tests(data)

In [8]:
def get_results(browser_tests):
    browsers = dict()
    
    for browser_name, browser in browser_tests.items():
        browsers[browser_name] = {
            "total_pass_count": 0,
            "total_fail_count": 0,
            "category": dict()
        }
        
        for category_name, category in browser.items():
            browsers[browser_name]["category"][category_name] = dict()
            browsers[browser_name]["category"][category_name]["pass_count"] = 0
            browsers[browser_name]["category"][category_name]["fail_count"] = 0
            for test_name, test in category.items():
                if test:
                    browsers[browser_name]["category"][category_name]["pass_count"] += 1
                else:
                    browsers[browser_name]["category"][category_name]["fail_count"] += 1
                
            browsers[browser_name]["total_pass_count"] += browsers[browser_name]["category"][category_name]["pass_count"]
            browsers[browser_name]["total_fail_count"] += browsers[browser_name]["category"][category_name]["fail_count"]

    sorted_browsers = list()
    browser_name2pass_count = dict()
    for browser_name, browser in browsers.items():
        browser_name2pass_count[browser_name] = browser["total_pass_count"]
    for browser_name, pass_count in sorted(browser_name2pass_count.items(), key=lambda item: item[1], reverse=True):
        sorted_browsers.append({browser_name: browsers[browser_name]})
        
    return sorted_browsers

In [9]:
def get_plot(data):
    
    browser_names = list()
    passed_tests = list()
    failed_test = list()
    amnt_tests = list()
    
    for ele in data:
        browser_name, browser = list(ele.items())[0]
        browser_names.append(browser_name)
        passed_tests.append(browser["total_pass_count"])
        failed_test.append(browser["total_fail_count"])
        amnt_tests.append(browser["total_pass_count"] + browser["total_fail_count"])
    
    source = ColumnDataSource(data={
        "x": list(range(len(browser_names))),
        "0": [0] * len(browser_names),
        "browser_names": browser_names,
        "passed_tests": passed_tests,
        "failed_tests": failed_test,
        "amnt_tests": amnt_tests
    })
    
    plot = figure(
        x_range=browser_names,
        y_range=Range1d(0, amnt_tests[0]),
        width=3840, 
        height=2160,
        title="Browser Privatsphäretests",
        tools="save"
    )
    
    # Dark Mode: Hintergrundfarben und Rahmen anpassen
    plot.background_fill_color = "#2d2d2d"
    plot.border_fill_color = "#2d2d2d"
    plot.outline_line_color = "#444444"
    
    # Achsen und Titel in heller Farbe
    plot.title.text_color = "#ffffff"
    plot.xaxis.axis_label_text_color = "#ffffff"
    plot.yaxis.axis_label_text_color = "#ffffff"
    plot.xaxis.major_label_text_color = "#ffffff"
    plot.yaxis.major_label_text_color = "#ffffff"
    
    # Balken für bestandene und durchgefallene Tests
    renderer_pass = plot.vbar(x="browser_names", width=0.5, bottom="0", top="passed_tests", source=source, color="green", line_color="black", legend_label="Bestanden")
    renderer_fail = plot.vbar(x="browser_names", width=0.5, bottom="passed_tests", top="amnt_tests", source=source, color="#e74c3c", line_color="black", legend_label="Durchgefallen/Nicht unterstützt")
    
    # Schriftgrößen und Beschriftung der X-Achse anpassen
    plot.title.text_font_size = "20pt"
    plot.title.align = "center"
    plot.xaxis.major_label_text_font_size = "15pt"
    plot.yaxis.major_label_text_font_size = "15pt"
    plot.xaxis.axis_label_text_font_size = "18pt"
    plot.yaxis.axis_label_text_font_size = "18pt"
    
    # Drehung der X-Achsen-Beschriftungen
    plot.xaxis.major_label_orientation = 0.5  # 0.5 entspricht etwa 45 Grad
    
    # Beschriftungen hinzufügen
    plot.yaxis.axis_label = "Anzahl Tests"
    plot.sizing_mode='scale_both'
    quote = f"Edelstein, Arthur. ({update_date}). PrivacyTests.org. Abgerufen am {quote_ts.strftime('%d.%m.%Y')}, von https://privacytests.org"
    plot.add_layout(Title(text=quote, align="left", text_font_size = "10pt", text_color = "#ffffff", text_font_style="normal"), "below")
    
    # HTML-Template für Tooltips
    html_tooltip_pass = """
    <div style="background-color: #2c3e50; color: #ecf0f1; font-weight: bold; padding: 10px; margin: -10px; border-radius: 5px;">
        <span style="font-size: 18px; font-weight: bold;">Browser:</span>
        <span style="font-size: 18px; color: #2ecc71;">@browser_names</span><br>
        <span style="font-size: 18px; font-weight: bold;">Bestanden:</span>
        <span style="font-size: 18px; font-weight: bold; color: #2ecc71;">@passed_tests</span>
    </div>
    """
    
    html_tooltip_fail = """
    <div style="background-color: #2c3e50; color: #ecf0f1; padding: 10px; margin: -10px; border-radius: 5px;">
        <span style="font-size: 18px; font-weight: bold;">Browser:</span>
        <span style="font-size: 18px; font-weight: bold; color: #e74c3c;">@browser_names</span><br>
        <span style="font-size: 18px; font-weight: bold;">Durchgefallen:</span>
        <span style="font-size: 18px; font-weight: bold; color: #e74c3c;">@failed_tests</span>
    </div>
    """
    
    # HoverTool mit HTML-Tooltips
    hover_tool_pass = HoverTool(
        renderers=[renderer_pass],
        tooltips=html_tooltip_pass,
        point_policy="follow_mouse"
    )
    
    hover_tool_fail = HoverTool(
        renderers=[renderer_fail],
        tooltips=html_tooltip_fail,
        point_policy="follow_mouse"
    )
    
    # Entferne alte Tools und füge neue hinzu
    plot.tools = [SaveTool()]
    plot.add_tools(hover_tool_pass, hover_tool_fail)
    plot.toolbar.logo = None

    return plot

In [10]:
results = get_results(browser_tests)
plot = get_plot(results)
show(plot)